In [ ]:
from funkcije import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
combined_data = load_data()

In [ ]:
combined_data['DatumPN'] = pd.to_datetime(combined_data['DatumPN'], format='%d.%m.%Y')
combined_data['Weekday'] = combined_data['DatumPN'].dt.day_name()
combined_data['Day'] = combined_data['DatumPN'].dt.day
combined_data['Month'] = combined_data['DatumPN'].dt.month
combined_data['Year'] = combined_data['DatumPN'].dt.year

In [ ]:
classification_colors = {
    'Z MATERIALNO ŠKODO': '#377eb8',  
    'Z LAŽJO TELESNO POŠKODBO': '#4daf4a', 
    'S HUDO TELESNO POŠKODBO': '#ff7f00',  
    'S SMRTNIM IZIDOM': '#e41a1c'  
}

In [ ]:
first_date = combined_data['DatumPN'].min()
last_date = combined_data['DatumPN'].max()

days = (last_date - first_date).days
weeks = days // 7
months = (last_date.year - first_date.year) * 12 + (last_date.month - first_date.month)
years = combined_data['Year'].nunique()
days, weeks, months, years 

# Najnevarnejši meseci

In [ ]:
unique_accidents = combined_data.groupby('ZaporednaStevilkaPN').first()

stacked_data = unique_accidents.groupby(['Month', 'KlasifikacijaNesrece']).size().unstack(fill_value=0)

ordered_columns = ['Z MATERIALNO ŠKODO', 'Z LAŽJO TELESNO POŠKODBO', 'S HUDO TELESNO POŠKODBO', 'S SMRTNIM IZIDOM']
stacked_data[ordered_columns] = stacked_data[ordered_columns] / years

stacked_data = stacked_data[ordered_columns]
stacked_data = stacked_data.reindex(
    range(12, 0, -1),
    axis=0
)

stacked_data.plot(
    kind='barh',
    stacked=True,
    color=[classification_colors.get(col, 'gray') for col in stacked_data.columns],
    figsize=(10, 5)
)

plt.title('Nesreče povprečno na mesec', fontsize=14)
plt.xlabel('Povprečno število nesreč', fontsize=12)
plt.ylabel('Mesec', fontsize=12)
plt.yticks(range(0, 12), ['Dec', 'Nov', 'Okt', 'Sep', 'Avg', 'Jul', 'Jun', 'Maj', 'Apr', 'Mar', 'Feb', 'Jan'])
plt.legend(title='Klasifikacija nesreč', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.tight_layout()
plt.show()

# Najnevarnejši dnevi v tednu

In [ ]:
unique_accidents = combined_data.groupby('ZaporednaStevilkaPN').first()

stacked_data = unique_accidents.groupby(['Weekday', 'KlasifikacijaNesrece']).size().unstack(fill_value=0)

ordered_columns = ['Z MATERIALNO ŠKODO', 'Z LAŽJO TELESNO POŠKODBO', 'S HUDO TELESNO POŠKODBO', 'S SMRTNIM IZIDOM']
stacked_data[ordered_columns] = stacked_data[ordered_columns] / weeks

stacked_data = stacked_data[ordered_columns]
stacked_data = stacked_data.reindex(
    ['Sunday', 'Saturday', 'Friday', 'Thursday', 'Wednesday', 'Tuesday', 'Monday'],
    axis=0
)

stacked_data.plot(
    kind='barh',
    stacked=True,
    color=[classification_colors.get(col, 'gray') for col in stacked_data.columns],
    figsize=(10, 5)
)

plt.title('Nesreče povprečno na dan v tednu', fontsize=14)
plt.xlabel('Povprečno število nesreč', fontsize=12)
plt.ylabel('Dan v tednu', fontsize=12)
plt.yticks(range(0, 7), ['Ned', 'Sob', 'Pet', 'Čet', 'Sre', 'Tor', 'Pon'])
plt.legend(title='Klasifikacija nesreč', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.tight_layout()
plt.show()

# Vpliv praznikov

In [ ]:
def get_easter_date(year):
    a = year % 19
    b = year // 100
    c = year % 100
    d = b // 4
    e = b % 4
    f = (b + 8) // 25
    g = (b - f + 1) // 3
    h = (19 * a + b - d - g + 15) % 30
    i = c // 4
    k = c % 4
    l = (32 + 2 * e + 2 * i - h - k) % 7
    m = (a + 11 * h + 22 * l) // 451
    month = (h + l - 7 * m + 114) // 31
    day = ((h + l - 7 * m + 114) % 31) + 1
    return month, day

In [ ]:
# https://www.gov.si/teme/drzavni-prazniki-in-dela-prosti-dnevi/
holidays = dict() # {'datum': 'praznik'}
def add_holiday(date, name):
    if date < first_date or date > last_date:
        return
    if date in holidays:
        if isinstance(holidays[date], list):
            holidays[date].append(name)
        else:
            holidays[date] = [holidays[date], name]
    else:
        holidays[date] = name

for year in range(first_date.year, last_date.year + 1):
    add_holiday(pd.to_datetime(f"{year}-01-01"), 'novo leto')
    add_holiday(pd.to_datetime(f"{year}-01-02"), 'novo leto')
    add_holiday(pd.to_datetime(f"{year}-02-08"), 'Prešernov dan, slovenski kulturni praznik')
    add_holiday(pd.to_datetime(f"{year}-04-27"), 'dan upora proti okupatorju')
    add_holiday(pd.to_datetime(f"{year}-05-01"), 'praznik dela')
    add_holiday(pd.to_datetime(f"{year}-05-02"), 'praznik dela')
    add_holiday(pd.to_datetime(f"{year}-06-08"), 'dan Primoža Trubarja')
    add_holiday(pd.to_datetime(f"{year}-06-25"), 'dan državnosti')
    add_holiday(pd.to_datetime(f"{year}-08-15"), 'Marijino vnebovzetje')
    add_holiday(pd.to_datetime(f"{year}-08-17"), 'združitev prekmurskih Slovencev z matičnim narodom')
    add_holiday(pd.to_datetime(f"{year}-09-15"), 'priključitev Primorske k matični domovini')
    add_holiday(pd.to_datetime(f"{year}-09-23"), 'dan slovenskega športa')
    add_holiday(pd.to_datetime(f"{year}-10-25"), 'dan suverenosti')
    add_holiday(pd.to_datetime(f"{year}-10-31"), 'dan reformacije')
    add_holiday(pd.to_datetime(f"{year}-11-01"), 'dan spomina na mrtve')
    add_holiday(pd.to_datetime(f"{year}-11-10"), 'dan znanosti')
    add_holiday(pd.to_datetime(f"{year}-11-23"), 'dan Rudolfa Maistra')
    add_holiday(pd.to_datetime(f"{year}-12-25"), 'božič')
    add_holiday(pd.to_datetime(f"{year}-12-26"), 'dan samostojnosti in enotnosti')

    month, day = get_easter_date(year)
    easter_date = pd.to_datetime(f"{year}-{month}-{day}")

    add_holiday(easter_date, 'velikonočna nedelja')
    add_holiday(easter_date + pd.Timedelta(days=1), 'velikonočni ponedeljek')
    add_holiday(easter_date + pd.Timedelta(days=49), 'binkoštna nedelja')

In [ ]:
def get_holiday_occurence_count(holiday):
    if holiday == 'ni praznik':
        return days - len(holidays)
    i = 0
    for date in holidays:
        if isinstance(holidays[date], list):
            if holiday in holidays[date]:
                i += 1
        else:
            if holidays[date] == holiday:
                i += 1
    return i

In [ ]:
holiday_accidents = combined_data.copy()
holiday_accidents['Holiday'] = holiday_accidents['DatumPN'].map(holidays)
holiday_accidents = holiday_accidents.explode('Holiday') # 2014-06-08 je dan Rudolfa Maistra in binkoštna nedelja
holiday_accidents['Holiday'] = holiday_accidents['Holiday'].fillna('ni praznik')
holiday_accidents = holiday_accidents.groupby('ZaporednaStevilkaPN').first()

stacked_data = holiday_accidents.groupby(['Holiday', 'KlasifikacijaNesrece'], dropna=False).size().unstack(fill_value=0)

ordered_columns = ['Z MATERIALNO ŠKODO', 'Z LAŽJO TELESNO POŠKODBO', 'S HUDO TELESNO POŠKODBO', 'S SMRTNIM IZIDOM']
for col in ordered_columns:
    stacked_data[col] = stacked_data.apply(
        lambda row: row[col] / get_holiday_occurence_count(row.name),
        axis=1
    )

stacked_data = stacked_data.loc[stacked_data.sum(axis=1).sort_values(ascending=False).index]

stacked_data.plot(
    kind='barh',
    stacked=True,
    color=[classification_colors.get(col, 'gray') for col in stacked_data.columns],
    figsize=(10, 5)
)

plt.title('Nesreče povprečno na praznik', fontsize=14)
plt.xlabel('Povprečno število nesreč', fontsize=12)
plt.ylabel('Praznik', fontsize=12)

plt.legend(title='Klasifikacija nesreč', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
work_free_holidays = dict()
work_free_holidays['novo leto']                                             = True
work_free_holidays['Prešernov dan, slovenski kulturni praznik']             = True
work_free_holidays['velikonočna nedelja']                                   = True
work_free_holidays['velikonočni ponedeljek']                                = True
work_free_holidays['dan upora proti okupatorju']                            = True
work_free_holidays['praznik dela']                                          = True
work_free_holidays['binkoštna nedelja']                                     = True
work_free_holidays['dan Primoža Trubarja']                                  = False
work_free_holidays['dan državnosti']                                        = True
work_free_holidays['Marijino vnebovzetje']                                  = True
work_free_holidays['združitev prekmurskih Slovencev z matičnim narodom']    = False
work_free_holidays['priključitev Primorske k matični domovini']             = False
work_free_holidays['dan slovenskega športa']                                = False
work_free_holidays['dan suverenosti']                                       = False
work_free_holidays['dan reformacije']                                       = True
work_free_holidays['dan spomina na mrtve']                                  = True
work_free_holidays['dan znanosti']                                          = False
work_free_holidays['dan Rudolfa Maistra']                                   = False
work_free_holidays['božič']                                                 = True
work_free_holidays['dan samostojnosti in enotnosti']                        = True
work_free_holidays['ni praznik']                                            = False

work_free_count = list(work_free_holidays.values()).count(True)
work_free_count # dela prostih dni na leto


In [ ]:
def get_work_free_occurence_count(name):
    if name == True:
        return years * work_free_count
    else:
        return days - years * work_free_count

In [ ]:
work_free_accidents = combined_data.copy()
work_free_accidents['Holiday'] = work_free_accidents['DatumPN'].map(holidays)
work_free_accidents = work_free_accidents.explode('Holiday') # 2014-06-08 je dan Rudolfa Maistra in binkoštna nedelja
work_free_accidents['WorkFree'] = work_free_accidents['Holiday'].map(work_free_holidays)
work_free_accidents['WorkFree'] = work_free_accidents['WorkFree'].fillna(False)
work_free_accidents = work_free_accidents.groupby('ZaporednaStevilkaPN').first()

stacked_data = work_free_accidents.groupby(['WorkFree', 'KlasifikacijaNesrece']).size().unstack(fill_value=0)

ordered_columns = ['Z MATERIALNO ŠKODO', 'Z LAŽJO TELESNO POŠKODBO', 'S HUDO TELESNO POŠKODBO', 'S SMRTNIM IZIDOM']
for col in ordered_columns:
    stacked_data[col] = stacked_data.apply(
        lambda row: row[col] / get_work_free_occurence_count(row.name),
        axis=1
    )

stacked_data = stacked_data[ordered_columns]
stacked_data = stacked_data.reindex(
    [True, False],
    axis=0
)

stacked_data.plot(
    kind='barh',
    stacked=True,
    color=[classification_colors.get(col, 'gray') for col in stacked_data.columns],
    figsize=(10, 5)
)

plt.title('Nesreče povprečno na dela prost dan', fontsize=14)
plt.xlabel('Povprečno število nesreč', fontsize=12)
plt.ylabel('Dela prost dan', fontsize=12)
plt.yticks(range(0, 2), ['Je', 'Ni'])
plt.legend(title='Klasifikacija nesreč', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.tight_layout()
plt.show()

# Povprečno število nesreč na dan skozi čas

In [ ]:
def plot_rolling_average(ax, window, title):
    ax.set_ylim([0, 70])

    for col in ordered_columns:
        ax.plot(
            stacked_data.index,
            stacked_data[col].rolling(window=window, center=True, min_periods=1).mean(),
            label=col,
            color=classification_colors.get(col, 'gray')
        )

    ax.set_title(title, fontsize=14)
    ax.set_xlabel('Datum', fontsize=12)
    ax.set_ylabel('Povprečno število nesreč na dan', fontsize=12)
    ax.legend(title='Klasifikacija nesreč', fontsize=10)


In [ ]:
unique_accidents = combined_data.groupby('ZaporednaStevilkaPN').first()

stacked_data = unique_accidents.groupby(['DatumPN', 'KlasifikacijaNesrece']).size().unstack(fill_value=0)
ordered_columns = ['Z MATERIALNO ŠKODO', 'Z LAŽJO TELESNO POŠKODBO', 'S HUDO TELESNO POŠKODBO', 'S SMRTNIM IZIDOM']
stacked_data = stacked_data[ordered_columns]
stacked_data = stacked_data.sort_index()  

fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(10, 10))

plot_rolling_average(ax[0], window=365, title='Nesreče povprečno na dan, povprečje zadnjih 365 dni')
plot_rolling_average(ax[1], window=100, title='Nesreče povprečno na dan, povprečje zadnjih 100 dni')